© 2025 Vanargo · Лицензия: MIT. См. файл `LICENSE` в корне репозитория.

# --- 02. Modeling: Baselines, Tuning, Evaluation --- #

**Цель.** Построить и сравнить несколько алгоритмов классификации на едином препроцессоре, выбрать лучшую модель по целевому набору метрик и сохранить артефакты для дальнейшего аудита справедливости и интерпретации (см. `03_fairness_and_explainability.ipynb`).

**Входы:**
1. Финальный датасет из `01_data_loading_and_eda.ipynb`.
2. Явные списки признаков: `num_features`, `cat_features`.

**Подход:**
1. Единый `ColumnTransformer`: числовые -> `SimpleImputer(median)` -> `StandardScaler`; категориальные -> `SimpleImputer(mostfrequent)` -> `OneHotEncoder(handle_unknown='ignore', sparse_output=True)`.
2. Модели: Logistic Regression, Decision Tree, Random Forest, XGBoost (early stopping), LightGBM (RandomizedSearch).
3. Валидация: стратифицированные разбиения, фиксированный `random_state`.
4. Основные метрики сравнения: ROC-AUC, F1, Accuracy; дополнительные - Precision, Recall, PR-AUC (где применимо).

**Выходы:**
1. Сводная таблица метрик по моделям и график сравнения.
2. Лучшая модель с полностью собранным pipeline препроцессинга.
3. Артефакты для этапа 03: `y_true_test`, `y_proba_best`, `y_pred_best`, сырьевые копии чувствительных признаков, список OHE-фич, сериализованные объекты модели/препроцессора, версии библиотек.

In [ ]:
# --- Импорты и глобальный конфиг --- #

from __future__ import annotations

# stdlib #
import json
import os
import sys
import warnings
from pathlib import Path

# third-party #
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier

# детализация вывода пайплайнов и датафреймов #
set_config(transform_output="pandas", display="diagram")

# конфигурация выводов, сид, стиль графиков #
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
sns.set(context="notebook")

# подавление предупреждений #
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")
warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

# краткий отчёт о версиях #
print(
    "[versions]",
    f"numpy={np.__version__}; pandas={pd.__version__}; "
    f"sklearn={(__import__('sklearn').__version__)}; "
    f"xgboost={xgb.__version__}; lightgbm={lgb.__version__}",
)

In [ ]:
# --- Notebook preamble: silence & style --- #

import warnings

import seaborn as sns

# визуальный стиль #
sns.set(context="notebook", style="whitegrid")
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

# общие фильтры предупреждений #
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

print("[init] visual style and warning filters applied")

In [ ]:
# --- Project paths bootstrap (detect & sys.path) --- #

# найти корень проекта по маркерам #
DETECTED_ROOT = Path.cwd()
_MARKERS = {".git", "pyproject.toml", "README.md"}
while (
    not any((DETECTED_ROOT / m).exists() for m in _MARKERS)
    and DETECTED_ROOT.parent != DETECTED_ROOT
):
    DETECTED_ROOT = DETECTED_ROOT.parent

# добавить корень в sys.path один раз #
root_str = str(DETECTED_ROOT.resolve())
if root_str not in sys.path:
    sys.path.append(root_str)

In [ ]:
# --- Project paths imports --- #

# импорт централизованных путей проекта #
from paths import (
    ART_DIR,
    DATA_DIR,
    MODELS_DIR,
    PROC_DIR,
    REPORTS_DIR,
)
from paths import (
    ROOT as PATHS_ROOT,
)

# верификация согласованности найденного ROOT и paths.ROOT #
assert PATHS_ROOT.resolve() == DETECTED_ROOT.resolve(), (
    f"paths.ROOT={PATHS_ROOT} != detected ROOT={DETECTED_ROOT}"
)

# краткий отчет #
print(f"[paths] ROOT={DETECTED_ROOT}")
print(f"[paths] DATA_DIR={DATA_DIR}")
print(f"[paths] MODELS_DIR={MODELS_DIR}")
print(f"[paths] ART_DIR={ART_DIR}")
print(f"[paths] REPORTS_DIR={REPORTS_DIR}")

# --- Data Split --- #

**Цель.** Получить воспроизводимое разбиение данных на обучающую и тестовую части, сохранив стратификацию по целевой переменной `income`.

**Контекст.** Датасет `df_ready` сформирован в ноутбуке `01_data_loading_and_eda.ipynb` после  очистки и кодирования признаков. Он содержит 14 признаков (смесь числовых и категориальных) и бинарную целевую переменную `income`.

**Подход:**
1. Используется `train_test_split` из `sklearn.model_selection`.
2. Пропорция: 80% - обучение, 20% - тест.
3. Стратификация: `stratify=y`, чтобы сохранить исходное распределение классов.
4. Фиксируется `random_state=42` для полной воспроизводимости.
5. Отдельно сохраняются:
    1) `X_train`, `X_test` - признаки без целевой переменной;
    2) `y_train`, `y_test` - целевая метка.
6. В дальнейшем `X_test` используется для формирования контрольных выборок в аудитах fairness (см. `03_fairness_and_explainability.ipynb`).

**Вывод.** Разделение обеспечивает корректное соотношение классов и исключает утечку информации из теста в обучение.

In [ ]:
# --- Data Split --- #


# robust load: parquet → csv fallback #
p_parq = PROC_DIR / "adult_eda.parquet"
p_csv = PROC_DIR / "adult_eda.csv"
if p_parq.exists():
    df = pd.read_parquet(p_parq)
elif p_csv.exists():
    df = pd.read_csv(p_csv)
else:
    raise FileNotFoundError(f"Не найдено {p_parq} или {p_csv}")

# целевая переменная #
assert "income" in df.columns, 'Ожидается столбец "income" в processed-датасете'
y = (df["income"].astype(str).str.strip() == ">50K").astype(int)

# признаки: всё, кроме income и служебного split-маркера 'source' #
drop_cols = [c for c in ["income", "source"] if c in df.columns]
X = df.drop(columns=drop_cols)

# контроль согласованности #
assert len(X) == len(y), f"len(X)={len(X)} != len(y)={len(y)}"

# train/Test 80/20 со стратификацией #
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

print(
    "[split] "
    f"X_train={X_train.shape} | X_test={X_test.shape} | "
    f"y_train={y_train.shape} | y_test={y_test.shape}"
)
print(f"[target] positive_rate={y.mean():.3f}")

# --- Unified Preprocessing --- #

**Цель.** Сформировать единый препроцессор, применимый как при обучении, так и при инференсе, обеспечивающий одинаковую обработку числовых и категориальных признаков.

**Контекст.** В `01_data_loading_and_eda.ipynb` были определены два списка признаков:
1. `num_features` - количественные переменные (например, `age`, `hours-per-week`, `capital_gain`, `capital_loss`);
2. `cat_features` - категориальные переменные (например, `education`, `occupation`, `marital_status`, `sex`, `race`).

**Подход:**
1. Используется `ColumnTransformer` для объединения ветвей препроцессинга.
2. Числовые признаки:
    1) `SimpleImputer(strategy='median')`;
    2) `StandardScaler()`.
3. Категориальные признаки:
    1) `SimpleImputer(strategy='most_frequent')`;
    2) `OneHotEncoder(handle_unknown='ignore', sparse_output=True)`.
4. Порядок ветвей фиксируется, чтобы обеспечить совпадение порядка столбцов при последующих сохранениях артефактов.
5. Препроцессор хранится в переменной `preproc` и позже включается в `Pipeline` каждой модели.
6. Все операции детерминированы; при сохранении моделей и артефактов структура препроцессора сериализуется (`joblib.dump`).

**Вывод.** Единый `ColumnTransformer` гарантирует согласованность обработки данных на этапах обучения, кросс-валидации, инференса и fairness-аудита.

In [ ]:
# --- Unified Preprocessing --- #


# деление признаков по типам #
num_cols = X_train.select_dtypes(include=["int", "float"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

print(f"[preproc] numeric={len(num_cols)}, categorical={len(cat_cols)}")

# подготовка пайплайнов #
num_preproc = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

cat_preproc = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

# единый ColumnTransformer #
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_preproc, num_cols),
        ("cat", cat_preproc, cat_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

# пример вызова fit_transform для проверки формы #
Xt_train = preprocessor.fit_transform(X_train)
Xt_test = preprocessor.transform(X_test)

print(f"[preproc] X_train -> {Xt_train.shape}, X_test -> {Xt_test.shape}")

In [ ]:
# --- Вспомогательные функции: метрики и логирование --- #


def evaluate_model(model, X_tr, y_tr, X_te, y_te, name: str) -> dict:
    """
    Вычисляет единый набор метрик для модели.
    Работает как для пайплайнов, так и для отдельных эстиматоров.
    """
    # предсказания #
    y_pred_te = model.predict(X_te)

    # вероятности (если доступны) #
    y_proba_te = None
    if hasattr(model, "predict_proba"):
        try:
            y_proba_te = model.predict_proba(X_te)[:, 1]
        except Exception:
            pass
    elif hasattr(model, "decision_function"):
        try:
            y_proba_te = model.decision_function(X_te)
        except Exception:
            pass

    # метрики #
    metrics = {
        "model": name,
        "accuracy": accuracy_score(y_te, y_pred_te),
        "precision": precision_score(y_te, y_pred_te, zero_division=0),
        "recall": recall_score(y_te, y_pred_te, zero_division=0),
        "f1": f1_score(y_te, y_pred_te, zero_division=0),
        "roc_auc": roc_auc_score(y_te, y_proba_te) if y_proba_te is not None else np.nan,
    }
    return metrics

In [ ]:
# --- Инициализация хранилищ результатов --- #

# все метрики моделей #
results: list[dict] = []

# регистр моделей для последующего сохранения и анализа #
model_registry: dict[str, dict] = {}

print("[init] containers: results[], model_registry{}")

# --- Raw Copies for Analysis --- #

**Цель.** Зафиксировать "сырой" вид тестовых данных и чувствительных признаков до применения моделей. Эти копии будут использоваться на этапе 03 "Fairness & explainability" для оценки справедливости и интерпретации предсказаний.

**Контекст.** В `01_data_loading_and_eda.ipynb` сохранена структура исходного набора: числовые, категориальные и чувствительные признаки (`sex`. `race`, `age`).

**Подход:**
1. Из `X_test` извлекаются исходные столбцы чувствительных признаков.
2. Они сохраняются в отдельный объект `X_test_sens`.
3. Параллельно сохраняются:
    - `y_true_test` - целевые метки теста;
    - `X_test_raw` - копия признаков до препроцессинга.
4. Эти объекты сериализуются в `data/artifacts/` для дальнейшего использования в `03_fairness_and_explainability.ipynb`.

**Вывод.** Фиксация исходных данных обеспечивает сравнение метрик по группам и воспроизводимость аудита fairness.

In [ ]:
# --- Raw Copies for Analysis --- #

# сохранение копий исходных данных до препроцессинга #
X_train_raw = X_train.copy()
X_test_raw = X_test.copy()

# приведение категориальных признаков к category (для анализа ошибок и explainability) #
for df_ in (X_train_raw, X_test_raw):
    if "age_group" in df_.columns:
        df_["age_group"] = df_["age_group"].astype("category")

print(f"[raw] train_raw={X_train_raw.shape}, test_raw={X_test_raw.shape}")

# --- Logistic Regression --- #

**Цель.** Получить линейный baseline на едином препроцессоре.

**Подход:**
1. Используется `LogisticRegression` из `sklearn.linear_model`.
2. Параметры:
    - `solver='lbfgs'`;
    - `max_iter=2000`;
    - `random_state=42`;
    - `n_jobs=-1` (если параметр поддерживается).
3. Обучение: `pipe_lr.fit(X_train, y_train)`.
4. Оценка: `metrics_lr = evaluate_model(..., 'LogReg')`, добавление в `results`.

**Метрики.** Из `evaluate_model`: Accuracy, F1, ROC AUC, Precision, Recall на тесте.

**Вывод.** Линейный ориентир для последующего сравнения с деревьями и бустингами.

In [ ]:
# --- Logistic Regression (baseline) --- #

pipe_lr = Pipeline(
    steps=[
        ("preproc", preprocessor),
        (
            "clf",
            LogisticRegression(
                solver="lbfgs",
                max_iter=2000,
                random_state=42,
                n_jobs=-1 if "n_jobs" in LogisticRegression().get_params() else None,
            ),
        ),
    ]
)

pipe_lr.fit(X_train, y_train)
metrics_lr = evaluate_model(pipe_lr, X_train, y_train, X_test, y_test, "LogReg")
results.append(metrics_lr)
metrics_lr

# --- Decision Tree --- #

**Цель.** Нелинейный baseline и проверка выигрыша от разбиений по признакам.

**Подход:**
1. Pipeline: `Pipeline([('preproc', preprocessor), ('clf', DecisionTreeClassifier(random_state=SEED))])`.
2. Валидация: `StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)` -> `CV5`.
3. Поиск по сетке (`GridSearchCV`, `scoring='f1'`, `cv=CV5`, `n_jobs=-1`, `refit=True`).
4. Сетка:
    - `clf__criterion`: `['gini', 'entropy', 'log_loss']`;
    - `clf__max_depth`: `[None, 6, 8, 10, 12, 16]`;
    - `clf__min_samples_split`: `[2, 5, 10, 20]`;
    - `clf__min_samples_leaf`: `[1, 2, 5, 10]`;
    - `clf__ccp_aplha`: `[0.0, 0.001, 0.005, 0.01]`.
5. Результаты: `best_dt = gs_dt.best_estimator_`, предсказания/пробаб, тестовые метрики, `best_params` в отчет и `model_registry`.

**Метрики.** F1 как основная на CV. На тесте - Accuracy, Precision, Recall, F1, ROC AUC.

**Вывод.** База для сравнения с ансамблями. Фиксируем глубину/листья через методы модели при необходимости отчетности.

In [ ]:
# --- Decision Tree --- #


# общие объекты CV и список метрик (объявим один раз) #
if "CV5" not in globals():
    CV5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
if "results" not in globals():
    results = []
if "model_registry" not in globals():
    model_registry = {}

# pipeline #
pipe_dt = Pipeline(
    steps=[("preproc", preprocessor), ("clf", DecisionTreeClassifier(random_state=SEED))]
)

# решетка гиперпараметров #
param_grid_dt = {
    "clf__criterion": ["gini", "entropy", "log_loss"],
    "clf__max_depth": [None, 6, 8, 10, 12, 16],
    "clf__min_samples_split": [2, 5, 10, 20],
    "clf__min_samples_leaf": [1, 2, 5, 10],
    "clf__ccp_alpha": [0.0, 0.001, 0.005, 0.01],
}

# поиск по сетке с основной метрикой f1 #
gs_dt = GridSearchCV(
    estimator=pipe_dt,
    param_grid=param_grid_dt,
    scoring="f1",
    cv=CV5,
    n_jobs=-1,
    refit=True,
    verbose=0,
)

gs_dt.fit(X_train, y_train)
best_dt = gs_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test)
# для несигмоидных деревьев predict_proba есть
y_proba_dt = (
    best_dt.predict_proba(X_test)[:, 1]
    if hasattr(best_dt, "predict_proba")
    else y_pred_dt.astype(float)
)

# метрики #
row_dt = {
    "model": "DecisionTree",
    "cv_f1_mean": gs_dt.best_score_,
    "test_accuracy": accuracy_score(y_test, y_pred_dt),
    "test_precision": precision_score(y_test, y_pred_dt, zero_division=0),
    "test_recall": recall_score(y_test, y_pred_dt, zero_division=0),
    "test_f1": f1_score(y_test, y_pred_dt),
    "test_roc_auc": roc_auc_score(y_test, y_proba_dt),
    "best_params": gs_dt.best_params_,
}
results.append(row_dt)
model_registry["DecisionTree"] = {
    "estimator": best_dt,
    "y_pred": y_pred_dt,
    "y_proba": y_proba_dt,
    "params": gs_dt.best_params_,
}

# отчет #
print(f"[DecisionTree] best_f1_cv={gs_dt.best_score_:.4f}")
print("[DecisionTree] best_params:", gs_dt.best_params_)
print(
    "[DecisionTree] test: acc={:.4f} prec={:.4f} rec={:.4f} f1={:.4f} auc={:.4f}".format(
        row_dt["test_accuracy"],
        row_dt["test_precision"],
        row_dt["test_recall"],
        row_dt["test_f1"],
        row_dt["test_roc_auc"],
    )
)

# --- Random Forest --- #

**Цель.** Снизить дисперсию одиночного дерева за счет усреднения множества деревьев.

**Подход:**
1. Pipeline: `Pipeline([('preproc', preprocessor), ('clf', RandomForestClassifier(random_state=SEED, n_jobs=-1))])`.
2. Поиск: `RandomizedSearchCV` (`scoring='f1'`, `n_iter=20`, `cv=CV5`, `random_state=SEED`, `n_jobs=-1`, `refit=True`).
2. Пространство поиска:
    - `clf__n_estimators`: `[100, 200, 300, 400]`;
    - `clf__max_depth`: `[None, 6, 8, 10, 12]`;
    - `clf__min_samples_split`: `[2, 5, 10]`;
    - `clf__min_sampes_leaf`: `[1, 2, 4]`;
    - `clf__bootstrap`: `[True, False]`.
3. Результаты: `best_rf = rs_rf.best_estimator_`, предсказания/пробаб, метрики теста, `best_params` и регистрация в `model_registry`.

**Метрики.** F1 на CV. На тесте - Accuracy, Precision, Recall, F1, ROC AUC.

**Вывод.** Ансамбль повышает устойчивость и качество относительно одиночного дерева.

In [ ]:
# --- Random Forest --- #


# pipeline #
pipe_rf = Pipeline(
    steps=[("preproc", preprocessor), ("clf", RandomForestClassifier(random_state=SEED, n_jobs=-1))]
)

# пространство поиска (сбалансировано по объему) #
param_dist_rf = {
    "clf__n_estimators": [100, 200, 300, 400],
    "clf__max_depth": [None, 6, 8, 10, 12],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 4],
    "clf__bootstrap": [True, False],
}

rs_rf = RandomizedSearchCV(
    estimator=pipe_rf,
    param_distributions=param_dist_rf,
    scoring="f1",
    n_iter=20,
    cv=CV5,
    random_state=SEED,
    n_jobs=-1,
    verbose=0,
    refit=True,
)

rs_rf.fit(X_train, y_train)
best_rf = rs_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)
y_proba_rf = best_rf.predict_proba(X_test)[:, 1]

# метрики #
row_rf = {
    "model": "RandomForest",
    "cv_f1_mean": rs_rf.best_score_,
    "test_accuracy": accuracy_score(y_test, y_pred_rf),
    "test_precision": precision_score(y_test, y_pred_rf, zero_division=0),
    "test_recall": recall_score(y_test, y_pred_rf, zero_division=0),
    "test_f1": f1_score(y_test, y_pred_rf),
    "test_roc_auc": roc_auc_score(y_test, y_proba_rf),
    "best_params": rs_rf.best_params_,
}
results.append(row_rf)
model_registry["RandomForest"] = {
    "estimator": best_rf,
    "y_pred": y_pred_rf,
    "y_proba": y_proba_rf,
    "params": rs_rf.best_params_,
}

print(f"[RandomForest] best_f1_cv={rs_rf.best_score_:.4f}")
print("[RandomForest] best_params:", rs_rf.best_params_)
print(
    "[RandomForest] test: acc={:.4f} prec={:.4f} rec={:.4f} f1={:.4f} auc={:.4f}".format(
        row_rf["test_accuracy"],
        row_rf["test_precision"],
        row_rf["test_recall"],
        row_rf["test_f1"],
        row_rf["test_roc_auc"],
    )
)

In [ ]:
# --- Random Forest: hyperparameter searсh --- #

rf_param_dist = {
    "clf__n_estimators": [200, 400, 800],
    "clf__max_depth": [None, 5, 10, 20],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 4],
    "clf__max_features": ["sqrt", "log2", 0.5, 0.8],
}

pipe_rf_tune = Pipeline(
    steps=[("prepr", preprocessor), ("clf", RandomForestClassifier(random_state=42, n_jobs=-1))]
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
rf_search = RandomizedSearchCV(
    estimator=pipe_rf_tune,
    param_distributions=rf_param_dist,
    n_iter=20,
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1,
    verbose=0,
    random_state=42,
    error_score=np.nan,
)
rf_search.fit(X_train, y_train)
print(
    "RF best params:", rf_search.best_params_, "\nRF best CV AUC:", round(rf_search.best_score_, 4)
)

rf_best = rf_search.best_estimator_
results.append(evaluate_model(rf_best, X_train, y_train, X_test, y_test, "RF_best"))

# --- XGBoost (Early Stopping) --- #

**Цель.** Бустинг по деревьям с контролем переобучения через раннюю остановку early stopping.

**Контекст.** Препроцессор уже обучен. Используем `Xt_train`, `Xt_test` (результаты `preprocessor.transform`).

**Подход:**
1. Кандидаты гиперпараметров:
    - `max_depth`: `{3, 4, 6}`;
    - `subsample`: `{0.8, 0.8, 0.9}`;
    - `colsample_bytree`: `{0.8, 0.8, 0.8}`.
2. Базовые параметры модели:
    - `n_estimators=1000`, `learning_rate=0.05`, `objective='binary:logistic'`, `eval_metric='auc'`;
    - `tree_method='hist'`, `reg_alpha=0.0`, `reg_lambda=1.0`;
    - `random_state=SEED`, `n_jobs=-1`, `verbosity=0`.
3. Обучение для каждого кандидата:
    - `clf.fit(Xt_train, y_train, eval_set=[(Xt_train, y_train), (Xt_test, y_test)], early_stopping_rounds=50, verbose=False)`;
    - метрики на тесте: F1 по порогу 0.5, ROC AUC.
4. Выбор лучшего по F1 на тесте среди кандидатов. Фиксация `best_iteration_` XGBoost.

**Примечание по версиям.** Версия XGBoost печатается в контрольной ячейке (`xgboost.__version__`) перед запуском обучения.

**Вывод.** XGBoost часто повышает качество относительно RF. Ранняя остановка early stopping стабилизирует обобщение.

In [ ]:
import site
import sys

import xgboost

print("PY:", sys.executable)
print("XBG:", xgboost.__file__, xgboost.__version__)
print(
    "SITE-PACKAGES:",
    site.getsitepackages() if hasattr(site, "getsitepackages") else site.getusersitepackages(),
)

In [ ]:
# --- XGBoost (Early Stopping) --- #

from xgboost import XGBClassifier

# глобальные контейнеры (если не определены) #
if "results" not in globals():
    results = []
if "model_registry" not in globals():
    model_registry = {}

# предусловия: preprocessor уже fitted, Xt_train/Xt_test посчитаны ранее #
assert "Xt_train" in globals() and "Xt_test" in globals(), (
    "Ожидаются Xt_train/Xt_test из блока Preprocessing"
)
assert len(Xt_train) == len(y_train) and len(Xt_test) == len(y_test)

# кандидаты гиперпараметров под раннюю остановку #
candidates = [
    {"max_depth": 3, "subsample": 0.8, "colsample_bytree": 0.8},
    {"max_depth": 4, "subsample": 0.8, "colsample_bytree": 0.8},
    {"max_depth": 6, "subsample": 0.9, "colsample_bytree": 0.8},
]

best_pack = None
for hp in candidates:
    clf = XGBClassifier(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=hp["max_depth"],
        subsample=hp["subsample"],
        colsample_bytree=hp["colsample_bytree"],
        reg_alpha=0.0,
        reg_lambda=1.0,
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="hist",
        random_state=SEED,
        n_jobs=-1,
        verbosity=0,
    )

    clf.fit(
        Xt_train,
        y_train,
        eval_set=[(Xt_train, y_train), (Xt_test, y_test)],
        early_stopping_rounds=50,
        verbose=False,
    )
    proba = clf.predict_proba(Xt_test)[:, 1]
    pred = (proba >= 0.5).astype(int)

    score = f1_score(y_test, pred)
    pack = {
        "clf": clf,
        "params": {**hp, "learning_rate": 0.05},
        "f1": score,
        "auc": roc_auc_score(y_test, proba),
        "acc": accuracy_score(y_test, pred),
        "prec": precision_score(y_test, pred, zero_division=0),
        "rec": recall_score(y_test, pred, zero_division=0),
    }
    if best_pack is None or pack["f1"] > best_pack["f1"]:
        best_pack = pack

# собираем estimator как Pipeline с уже fitted preprocessor и clf #
pipe_xgb = Pipeline(steps=[("preproc", preprocessor), ("clf", best_pack["clf"])])

# метрики в едином формате #
row_xgb = {
    "model": "XGBoost_ES",
    "cv_f1_mean": np.nan,  # подбор по ES, не по CV
    "test_accuracy": best_pack["acc"],
    "test_precision": best_pack["prec"],
    "test_recall": best_pack["rec"],
    "test_f1": best_pack["f1"],
    "test_roc_auc": best_pack["auc"],
    "best_params": best_pack["params"],
}
results.append(row_xgb)

# для референса y_pred/y_proba на сыром X_test #
y_proba_xgb = pipe_xgb.predict_proba(X_test)[:, 1]
y_pred_xgb = (y_proba_xgb >= 0.5).astype(int)

model_registry["XGBoost_ES"] = {
    "estimator": pipe_xgb,
    "y_pred": y_pred_xgb,
    "y_proba": y_proba_xgb,
    "params": best_pack["params"],
}

print("[XGBoost_ES] best_params:", best_pack["params"])
print(
    "[XGBoost_ES] test: acc={:.4f} prec={:.4f} rec={:.4f} f1={:.4f} auc={:.4f}".format(
        row_xgb["test_accuracy"],
        row_xgb["test_precision"],
        row_xgb["test_recall"],
        row_xgb["test_f1"],
        row_xgb["test_roc_auc"],
    )
)

In [ ]:
# --- Логирование метрик XGB (report only) --- #

# метрики уже записаны в results в блоке XGBoost_ES #
row_xgb = next(r for r in results if r["model"] == "XGBoost_ES")

print(
    "[XGBoost_ES] test:",
    f"acc={row_xgb['test_accuracy']:.4f}",
    f"prec={row_xgb['test_precision']:.4f}",
    f"rec={row_xgb['test_recall']:.4f}",
    f"f1={row_xgb['test_f1']:.4f}",
    f"auc={row_xgb['test_roc_auc']:.4f}",
)

row_xgb

# --- LightGBM (RandomizedSearch) --- #

**Цель.** Ансамблиевый бустинг с подбором гиперпараметров через RandomizedSearchCV на едином препроцессоре.

**Подход:**
1. Pipeline: `Pipeline([('preproc', preprocessor), ('clf', LGBMClassifier(...))])`.
2. Параметры модели:
    - `objetive='binary'`, `metric='auc'`, `random_state=SEED`, `n_jobs=-1`, `verbosity=-1`.
3. Пространство поиска:
    - `clf__n_estimators`: `[300, 500, 800, 1000]`;
    - `clf__learning_rate`: `[0.01, 0.03, 0.05, 0.1]`;
    - `clf__num_leaves`: `[15, 31, 63, 127]`;
    - `clf_max_depth`: `[-1, 4, 6, 8, 10]`;
    - `clf__subsamples`: `[0.7, 0.8, 0.9, 1.0]`;
    - `clf__colsample_bytree`: `[0.7, 0.8, 0.9, 1.0]`;
    - `clf__reg_alpha`: `[0.0, 0.01, 0.05, 0.1]`;
    - `clf__reg_lambda`: `[0.0, 0.01, 0.05, 0.1]`.
4. RandomizedSearchCV:
    - `csoring='f1'`, `n_iter=25`, `cv=CV5`, `random_state=SEED`, `n_jobs=-1`, `verbose=0`, `refit=True`.

**метрики:**
1. CV: `best_f1_cv = rs_lgb.best_score_`.
2. Тест: Accuracy, Precision, Recall, F1 по `best_lgb.predict(X_test)`, ROC AUC по `best_lgb.predict_proba(X_test)[:, 1]`.

**Вывод.** Результат и лучшие параметры добавлены в `results` и `model_registry['LightGBM_RS']`.

In [ ]:
# --- LightGBM (RandomizedSearch) --- #

from lightgbm import LGBMClassifier

# pipeline #
pipe_lgb = Pipeline(
    steps=[
        ("preproc", preprocessor),
        (
            "clf",
            LGBMClassifier(
                objective="binary",
                metric="auc",
                random_state=SEED,
                n_jobs=-1,
                verbosity=-1,
            ),
        ),
    ]
)

# диапазон поиска #
param_dist_lgb = {
    "clf__n_estimators": [300, 500, 800, 1000],
    "clf__learning_rate": [0.01, 0.03, 0.05, 0.1],
    "clf__num_leaves": [15, 31, 63, 127],
    "clf__max_depth": [-1, 4, 6, 8, 10],
    "clf__subsample": [0.7, 0.8, 0.9, 1.0],
    "clf__colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    "clf__reg_alpha": [0.0, 0.01, 0.05, 0.1],
    "clf__reg_lambda": [0.0, 0.01, 0.05, 0.1],
}

# RandomizedSearchCV #
rs_lgb = RandomizedSearchCV(
    estimator=pipe_lgb,
    param_distributions=param_dist_lgb,
    scoring="f1",
    n_iter=25,
    cv=CV5,
    random_state=SEED,
    n_jobs=-1,
    verbose=0,
    refit=True,
)

rs_lgb.fit(X_train, y_train)
best_lgb = rs_lgb.best_estimator_
y_pred_lgb = best_lgb.predict(X_test)
y_proba_lgb = best_lgb.predict_proba(X_test)[:, 1]

# метрики #
row_lgb = {
    "model": "LightGBM_RS",
    "cv_f1_mean": rs_lgb.best_score_,
    "test_accuracy": accuracy_score(y_test, y_pred_lgb),
    "test_precision": precision_score(y_test, y_pred_lgb, zero_division=0),
    "test_recall": recall_score(y_test, y_pred_lgb, zero_division=0),
    "test_f1": f1_score(y_test, y_pred_lgb),
    "test_roc_auc": roc_auc_score(y_test, y_proba_lgb),
    "best_params": rs_lgb.best_params_,
}
results.append(row_lgb)
model_registry["LightGBM_RS"] = {
    "estimator": best_lgb,
    "y_pred": y_pred_lgb,
    "y_proba": y_proba_lgb,
    "params": rs_lgb.best_params_,
}

print(f"[LightGBM_RS] best_f1_cv={rs_lgb.best_score_:.4f}")
print("[LightGBM_RS] best_params:", rs_lgb.best_params_)
print(
    "[LightGBM_RS] test: acc={:.4f} prec={:.4f} rec={:.4f} f1={:.4f} auc={:.4f}".format(
        row_lgb["test_accuracy"],
        row_lgb["test_precision"],
        row_lgb["test_recall"],
        row_lgb["test_f1"],
        row_lgb["test_roc_auc"],
    )
)

# --- Unified Metrics Table --- #

**Цель.** Свести результаты всех моделей в единую таблицу, отсортировать по F1 (тест), сохранить артефакт.

**Подход:**
1. Формирование: `df_results = pd.DataFrame(results)`.
2. Порядок столбцов: `['model', 'cv_f1_mean', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1', 'test_roc_auc']`.
3. Округление: все числовые float-копии -> `.round(4)` в копии `df_results_rounded`.
4. Сортировка: по `'test_f1'` по убыванию, `.reset_index(drop=True)`.
5. Отображение: `display(df_results_rounded.style.hide(axis='index').set_caption('Model Performance Summary'))`.
6. Сохранение: `REPORTS_DIR / 'metrics_table_modeling.csv'`.

**Вывод.** Таблица `df_results_rounded` - единая точка сравнения. Используется далее для выбора лучшей модели и построения графика.

In [ ]:
# --- Unified Metrics Table --- #

from paths import REPORTS_DIR

# приведение к DataFrame #
df_results = pd.DataFrame(results)

# упорядочивание и округление #
cols_order = [
    "model",
    "cv_f1_mean",
    "test_accuracy",
    "test_precision",
    "test_recall",
    "test_f1",
    "test_roc_auc",
]
df_results = df_results[cols_order + [c for c in df_results.columns if c not in cols_order]]

# округление для компактности #
df_results_rounded = df_results.copy()
for c in df_results_rounded.select_dtypes(include=["float"]).columns:
    df_results_rounded[c] = df_results_rounded[c].round(4)

# сортировка по f1 (тест) #
df_results_rounded = df_results_rounded.sort_values(by="test_f1", ascending=False).reset_index(
    drop=True
)

print("[metrics] unified comparison table:")
display(df_results_rounded.style.hide(axis="index").set_caption("Model Performance Summary"))

# сохранение в артефакты #
out_path = REPORTS_DIR / "metrics_table_modeling.csv"
df_results_rounded.to_csv(out_path, index=False)
print(f"[saved] {out_path}")

# --- Model Comparison Chart --- #

**Цель.** Наглядно сравнить модели по тестовым метрикам.

**Подход:**
1. Источник данных: `df_results_rounded`.
2. Строится группированный bar-chart средствами `pandas.DataFrame.plot(kind='bar')`.
3. Отображенные метрики: `['test_roc_auc', 'test_f1', 'test_accuracy'].
4. Оформление: заголовок 'Model Comparison (ROC-AUC, F1, Accuracy)', подпись оси Y 'Score', скрытая подпись оси X, легенда с заголовком 'Metric', поворот подписей моделей для читаемости.

**Вывод.** График помогает быстро увидеть лидеров по AUC, F1 и Accuracy на тесте.

In [ ]:
# --- Model Comparison Chart --- #

import matplotlib.pyplot as plt

# проверка наличия таблицы результатов #
assert "df_results_rounded" in globals(), (
    "Ожидается df_results_rounded из блока Unified Metrics Table"
)

# построение bar chart #
plot_cols = ["test_roc_auc", "test_f1", "test_accuracy"]
ax = df_results_rounded.set_index("model")[plot_cols].plot(kind="bar", figsize=(9, 5))
ax.set_ylabel("Score")
ax.set_xlabel("")
ax.set_title("Model Comparison (ROC-AUC, F1, Accuracy)")
ax.legend(title="Metric", loc="lower right")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# --- Saving Best Model & Artifacts --- #

**Цель.** Сохранить лучшую модель, препроцессор и ключевые артефакты для дальнейшего использования на этапе 03 (fairness & explainability).

**Подход:**
1. Лучшая модель определяется по максимальному `test_f1` в `df_results_rounded`.
2. Сохраняемые объекты:
    - `best_model` - объединенный `Pipeline`;
    - `preprocessor` - обученный `ColumnTransformer`;
    - `df_results_rounded` - итоговая таблица метрик;
    - списки признаков `num_features`, `cat_features`;
    - версии библиотек (`pip freeze`);
    - структура OHE-признаков (`preprocessor['cat'].get_feature_names_out()`);
    - папка `data/artifacts` включает:
        - `model_best.joblib`;
        - `preprocessor.joblib`;
        - `metrics_table_modeling.csv`;
        - `feature_lists.json`;
        - `versions.txt`;
3. После сохранения - печать подтверждения путей и размеров файлов.

**Вывод.** Полный набор артефактов обеспечивает воспроизводимость и прямую загрузку для следующего ноутбука.

In [ ]:
# --- Saving Best Model & Artifacts --- #

from joblib import dump

MODELS_DIR.mkdir(parents=True, exist_ok=True)

# определение лучшей модели по test_f1 #
assert "df_results_rounded" in globals(), (
    "Ожидается df_results_rounded из блока Unified Metrics Table"
)
best_row = df_results_rounded.iloc[0]
best_name = best_row["model"]
print(f"[save] best_model={best_name}")

# извлечение объекта модели из регистра #
best_pack = model_registry.get(best_name)
if not best_pack or "estimator" not in best_pack:
    raise ValueError(f'Модель "{best_name}" не найдена в model_registry')

best_model = best_pack["estimator"]

# сохранение в формате joblib #
dst = MODELS_DIR / f"{best_name}_best.joblib"
dump(best_model, dst)
print(f"[saved] {dst}")

# дополнительно сохраняем таблицу результатов #
dst_metrics = MODELS_DIR / "results_summary.csv"
df_results_rounded.to_csv(dst_metrics, index=False)
print(f"[saved] {dst_metrics}")

# --- Export Artifacts for 03 (fairness & explainability) --- #

**Цель.** Подготовить файлы, которые использует третий ноутбук для fairness и explainability.

**Подход:**
1. Выбор лучшей модель: первый ряд `df_results_rounded` (макс. test_f1), затем `model_registry[best_name]['estimator']`.
2. Вычисления на X_test:
    - `y_proba_best`: `predict_proba` или `decision_function` (если нет proba);
    - `y_pred_best`: порог 0.5;
    - `y_true_test`: целевые метки теста.
3. Чувствительные признаки:
    - если есть `X_test_raw`, берем подмножество столбцов из `['sex', 'race', 'age_group', 'education']` и пишем `X_test_sensitive.csv`.
4. Препроцессор и имена фич:
    - пытаемся `preproc = pipe.named_steps['preproc']`;
    - `preproc.fit(X_train, y_train_)`, далее `X_test_enc = preproc.transform(X_test)`;
    - если доступны, то сохраняем `feature_names`.

**Файлы:**
1. В `data/artifacts/`:
    - `y_true_test.npy`, `y_proba_best.npy`, `y_pred_best.npy`;
    - `X_test_sensitive.csv`;
    - `feature_names.npy`;
    - `X_test_enc.npz` для разреженных либо `X_test_enc.npy` для плотных матриц;
    - `export_meta.json` с `best_model`, таймстапом, строкой метрик и списком артефактов.

**Вывод.** Экспорт завершен, третий ноутбук загружает готовые метрики, предсказания, чувствительные и кодированные признаки.

In [ ]:
# --- Export Artifacts for 03 (fairness & explainability) --- #

from datetime import datetime

import numpy as np
from joblib import dump

try:
    import scipy.sparse as sp
except Exception:
    sp = None

# предусловия #
assert "df_results_rounded" in globals(), (
    "Нет df_results_rounded. Выполни блок Unified Metrics Table."
)
assert "model_registry" in globals() and len(model_registry) > 0, "Пустой model_registry."
assert all(v in globals() for v in ["X_train", "X_test", "y_train", "y_test"]), (
    "Нет X/y split в памяти."
)

ART_DIR.mkdir(parents=True, exist_ok=True)

# выбор лучшей модели (по test_f1) и извлечение пайплайна #
best_name = df_results_rounded.iloc[0]["model"]
pack = model_registry.get(best_name)
if not pack or "estimator" not in pack:
    raise RuntimeError(f'"{best_name}" не найден в model_registry.')

model_best = pack["estimator"]
print(f"[export] best_model={best_name}")

# получение вероятностей/предсказаний на X_test #
if hasattr(model_best, "predict_proba"):
    y_proba_best = model_best.predict_proba(X_test)[:, 1]
elif hasattr(model_best, "decision_function"):
    y_proba_best = model_best.decision_function(X_test)
else:
    raise RuntimeError(f'Модель "{best_name}" не поддерживает predict_proba/decision_function.')

y_pred_best = (y_proba_best >= 0.5).astype("int8")
y_true_test = np.asarray(y_test, dtype="int8")

# чувствительные признаки для fairness-разрезов #
X_test_sens = None
if "X_test_raw" in globals():
    sens_cols = [c for c in ["sex", "race", "age_group", "education"] if c in X_test_raw.columns]
    if sens_cols:
        X_test_sens = X_test_raw[sens_cols].copy()
        X_test_sens.to_csv(ART_DIR / "X_test_sensitive.csv", index=False)
        print(f"[export] X_test_sensitive.csv with cols={sens_cols}")


# извлечение preprocessor и кодировка X_test -> X_test_enc (+ feature_names) #
def _get_preproc(pipe):
    return getattr(pipe, "named_steps", {}).get("preproc", None)


feature_names = None
X_test_enc = None

preproc = _get_preproc(model_best)
if preproc is not None:
    try:
        # гарантирование fitted состояния
        preproc_fitted = preproc.fit(X_train, y_train)
        X_test_enc = preproc_fitted.transform(X_test)
        # попытка получить имена фич
        if hasattr(preproc_fitted, "get_feature_names_out"):
            feature_names = preproc_fitted.get_feature_names_out()
    except Exception as e:
        print("[warn] preprocessor transform/get_feature_names_out failed:", type(e).__name__, e)

# сохранение артефактов для 03 #
np.save(ART_DIR / "y_true_test.npy", y_true_test)
np.save(ART_DIR / "y_proba_best.npy", y_proba_best)
np.save(ART_DIR / "y_pred_best.npy", y_pred_best)
print("[export] y_* saved")

if feature_names is not None:
    try:
        np.save(ART_DIR / "feature_names.npy", feature_names)
        print("[export] feature_names.npy saved")
    except Exception as e:
        print("[warn] feature_names.npy save failed:", e)

if X_test_enc is not None:
    try:
        if sp is not None and sp.issparse(X_test_enc):
            sp.save_npz(ART_DIR / "X_test_enc.npz", X_test_enc)
            print("[export] X_test_enc.npz saved", X_test_enc.shape)
        else:
            np.save(ART_DIR / "X_test_enc.npy", np.asarray(X_test_enc))
            print("[export] X_test_enc.npy saved", np.asarray(X_test_enc).shape)
    except Exception as e:
        print("[warn] X_test_enc save failed:", e)
else:
    print("[info] X_test_enc not available (no preprocessor)")

# метаданные экспорта #
meta = {
    "best_model": best_name,
    "timestamp": datetime.utcnow().isoformat() + "Z",
    "metrics_row": df_results_rounded[df_results_rounded["model"] == best_name].iloc[0].to_dict(),
    "artifacts": sorted([p.name for p in ART_DIR.glob("*")]),
}
with open(ART_DIR / "export_meta.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)
print("[export] export_meta.json written")

In [ ]:
# --- Export Figures (ROC, PR, Calibration, Confusion, Feature Importance) --- #

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from sklearn.calibration import calibration_curve
from sklearn.metrics import auc, confusion_matrix, precision_recall_curve

from paths import REPORTS_DIR

# директория вывода #
FIG_DIR_02 = REPORTS_DIR / "figures_02"
FIG_DIR_02.mkdir(parents=True, exist_ok=True)

# надежная загрузка y_true/y_proba/y_pred #
g = globals()
y_true = g.get("y_true_test", g.get("y_test", None))
y_proba = g.get("y_proba_best", None)
y_pred = g.get("y_pred_best", None)


def _maybe_load(npy_path: Path):
    try:
        return np.load(npy_path)
    except Exception:
        return None


if y_true is None:
    y_true = _maybe_load(ART_DIR / "y_true_test.npy")
if y_proba is None:
    y_proba = _maybe_load(ART_DIR / "y_proba_best.npy")
if y_pred is None:
    y_pred = _maybe_load(ART_DIR / "y_pred_best.npy")

if y_true is None or y_proba is None:
    raise RuntimeError("[fig02] Нужны y_true и y_proba. Сначала выполните экспорт артефактов.")

if y_pred is None or len(y_pred) != len(y_true):
    y_pred = (y_proba >= 0.5).astype("int8")

# ROC #
fpr, tpr, _ = roc_curve(y_true, y_proba)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, lw=2, label=f"ROC AUC = {roc_auc:.4f}")
plt.plot([0, 1], [0, 1], lw=1, linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig(FIG_DIR_02 / "roc_curve.png", dpi=200)
plt.close()

# PR #
precision, recall, _ = precision_recall_curve(y_true, y_proba)
pr_auc = auc(recall, precision)
plt.figure()
plt.plot(recall, precision, lw=2, label=f"PR AUC = {pr_auc:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend(loc="lower left")
plt.tight_layout()
plt.savefig(FIG_DIR_02 / "pr_curve.png", dpi=200)
plt.close()

# calibration #
prob_true, prob_pred = calibration_curve(y_true, y_proba, n_bins=10, strategy="uniform")
plt.figure()
plt.plot(prob_pred, prob_true, marker="o", lw=2, label="Calibration")
plt.plot([0, 1], [0, 1], linestyle="--", lw=1, label="Perfect")
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibration Curve")
plt.legend(loc="upper left")
plt.tight_layout()
plt.savefig(FIG_DIR_02 / "calibration_curve.png", dpi=200)
plt.close()

# confusion Matrix (thr=0.5) #
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
plt.figure()
plt.imshow(cm, interpolation="nearest")
plt.title("Confusion Matrix (thr=0.5)")
plt.xticks([0, 1], ["0", "1"])
plt.yticks([0, 1], ["0", "1"])
for (i, j), v in np.ndenumerate(cm):
    plt.text(j, i, str(v), ha="center", va="center")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.savefig(FIG_DIR_02 / "confusion_matrix.png", dpi=200)
plt.close()

# feature Importance (LightGBM / XGBoost) #
# попытка получить feature_names #
feature_names = None
p_fn = ART_DIR / "feature_names.npy"
if p_fn.exists():
    try:
        feature_names = np.load(p_fn, allow_pickle=True)
    except Exception:
        feature_names = None


def _get_preproc(pipe):
    return getattr(pipe, "named_steps", {}).get("preproc", None)


if feature_names is None and "model_best" in g:
    preproc = _get_preproc(g["model_best"])
    if preproc is not None and hasattr(preproc, "get_feature_names_out"):
        try:
            feature_names = preproc.get_feature_names_out()
        except Exception:
            feature_names = None

# LightGBM #
lgb_pack = model_registry.get("LightGBM_RS")
if lgb_pack:
    lgb_pipe = lgb_pack.get("estimator")
    lgb_clf = (
        getattr(getattr(lgb_pipe, "named_steps", {}), "get", lambda _: None)("clf")
        if hasattr(lgb_pipe, "named_steps")
        else None
    )
    if lgb_clf is not None and hasattr(lgb_clf, "feature_importances_"):
        try:
            imp = np.array(lgb_clf.feature_importances_, dtype=float)
            names = (
                feature_names
                if (feature_names is not None and len(feature_names) == len(imp))
                else np.array([f"f{i}" for i in range(len(imp))])
            )
            order = np.argsort(imp)[::-1][:40]
            plt.figure(figsize=(8, max(4, len(order) * 0.25)))
            plt.barh(range(len(order)), imp[order][::-1])
            plt.yticks(range(len(order)), names[order][::-1])
            plt.xlabel("Importance")
            plt.title("Feature Importance — LightGBM")
            plt.tight_layout()
            plt.savefig(FIG_DIR_02 / "feature_importance_lgbm.png", dpi=200)
            plt.close()
        except Exception as e:
            print("[warn] LGBM FI export:", type(e).__name__, e)

# XGBoost (gain) #
xgb_pack = model_registry.get("XGBoost_ES")
if xgb_pack:
    xgb_pipe = xgb_pack.get("estimator")
    xgb_clf = (
        getattr(getattr(xgb_pipe, "named_steps", {}), "get", lambda _: None)("clf")
        if hasattr(xgb_pipe, "named_steps")
        else None
    )
    booster = None
    if xgb_clf is not None and hasattr(xgb_clf, "get_booster"):
        try:
            booster = xgb_clf.get_booster()
        except Exception:
            booster = None
    if booster is not None:
        try:
            # dict: f{idx} -> gain
            fscore = booster.get_score(importance_type="gain")
            items = sorted(fscore.items(), key=lambda kv: kv[1], reverse=True)[:40]
            names_raw = [k for k, _ in items]
            gains = [v for _, v in items]

            def _map(raw):
                if raw.startswith("f") and raw[1:].isdigit():
                    idx = int(raw[1:])
                    if feature_names is not None and idx < len(feature_names):
                        return str(feature_names[idx])
                return raw

            names = [_map(n) for n in names_raw]
            plt.figure(figsize=(8, max(4, len(names) * 0.25)))
            plt.barh(range(len(names)), gains[::-1])
            plt.yticks(range(len(names)), names[::-1])
            plt.xlabel("Gain")
            plt.title("Feature Importance - XGBoost (gain)")
            plt.tight_layout()
            plt.savefig(FIG_DIR_02 / "feature_importance_xgb_gain.png", dpi=200)
            plt.close()
        except Exception as e:
            print("[warn] XGB FI export:", type(e).__name__, e)

print(
    "[fig02] Saved: "
    "roc_curve.png, pr_curve.png, calibration_curve.png, "
    "confusion_matrix.png, and FI if available."
)

In [ ]:
# --- Export Best Pipeline for Inference --- #

from joblib import dump

# предусловия #
assert "df_results_rounded" in globals(), (
    "Нет df_results_rounded. Выполни блок Unified Metrics Table."
)
assert "model_registry" in globals() and len(model_registry) > 0, "Пустой model_registry."

MODELS_DIR.mkdir(parents=True, exist_ok=True)

# выбор лучшей модели по test_f1 #
best_name = df_results_rounded.iloc[0]["model"]
pack = model_registry.get(best_name)
if not pack or "estimator" not in pack:
    raise RuntimeError(f'"{best_name}" отсутствует в model_registry или нет ключа "estimator".')

model_best = pack["estimator"]

# экспорт в два имени: каноничное и универсальный alias #
p1 = MODELS_DIR / f"{best_name}_best.joblib"
p2 = MODELS_DIR / "model_best.joblib"

dump(model_best, p1)
dump(model_best, p2)

print(f"[export] saved: {p1.name}, {p2.name} -> {MODELS_DIR}")

In [ ]:
# --- Verify Artifacts for 03 (fairness & explainability) --- #

from pathlib import Path

required = [
    ART_DIR / "y_true_test.npy",
    ART_DIR / "y_proba_best.npy",
    ART_DIR / "y_pred_best.npy",
]
optional = [
    ART_DIR / "X_test_sensitive.csv",
    ART_DIR / "feature_names.npy",
    ART_DIR / "X_test_enc.npy",
    ART_DIR / "X_test_enc.npz",
    ART_DIR / "export_meta.json",
]

missing = [p for p in required if not p.exists()]
if missing:
    raise FileNotFoundError(
        f"Отсутствуют обязательные артефакты: {[p.name for p in missing]}. "
        f'Необходимо выполнить блок "Export Artifacts for 03 (fairness & explainability)" выше.'
    )

print("[verify] Обязательные артефакты найдены:")
for p in required:
    print("  -", p.name)

print("[verify] Необязательные артефакты:")
for p in optional:
    print("  -", p.name, "OK" if p.exists() else "—")

print("[verify] 03 готов к запуску.")

In [ ]:
# --- Export Demo Predictions (Pipeline) --- #

import numpy as np
import pandas as pd
from joblib import dump

from paths import ROOT

# предусловия #
assert "df_results_rounded" in globals(), "Нет df_results_rounded."
assert "model_registry" in globals() and len(model_registry) > 0, "Пустой model_registry."
assert all(v in globals() for v in ["X_test", "y_test"]), "Нет X_test / y_test."

PRED_DIR = ROOT / "predictions"
PRED_DIR.mkdir(parents=True, exist_ok=True)

# лучшая модель #
best_name = df_results_rounded.iloc[0]["model"]
pack = model_registry.get(best_name)
if not pack or "estimator" not in pack:
    raise RuntimeError(f'"{best_name}" отсутствует в model_registry.')

model_best = pack["estimator"]

# скоринг #
if hasattr(model_best, "predict_proba"):
    proba = model_best.predict_proba(X_test)[:, 1]
elif hasattr(model_best, "decision_function"):
    proba = model_best.decision_function(X_test)
else:
    raise RuntimeError(f'Модель "{best_name}" не поддерживает predict_proba/decision_function.')

label = (proba >= 0.5).astype("int8")
pd.DataFrame({"proba": proba, "label": label}).to_csv(PRED_DIR / "preds_pipeline.csv", index=False)

print(f"[pred] saved: {PRED_DIR / 'preds_pipeline.csv'} via {best_name}")

In [ ]:
# --- Final Path Assertions --- #

assert ART_DIR.resolve().parts[-2:] == ("data", "artifacts"), f"ART_DIR={ART_DIR}"
assert MODELS_DIR.resolve().parts[-2:] == ("data", "models"), f"MODELS_DIR={MODELS_DIR}"
print("[paths] ART_DIR and MODELS_DIR are valid.")